# 8 元类编程

## 8.1 property动态属性

In [18]:
class User:
    def __init__(self, name, brithday):
        self.name = name
        self.brithday = brithday
        
    @property
    def info(self):
        return self.name+" "+self.brithday
    
    @info.setter
    def info(self, value):
        self.name = value
        
        


In [19]:
usera = User(name = "a", brithday="nba")
usera.info  # @property注解的方法可以直接当做属性来调用

'a nba'

In [20]:
usera.info = "asd"  # @info.setter 可为@property注解的属性设置setter函数
usera.info

'asd nba'

## 8.2 __getattr__与__getattribute__

In [21]:
class User:
    def __init__(self, name, brithday,info={}):
        self.name = name
        self.brithday = brithday
        
    def __getattr__(self, item):
        print("not find attr")

In [22]:
usera = User(name = "a", brithday="nba")
usera.i   # 在找不到属性时会调用__getattr__

not find attr


In [35]:
class Users:
    def __init__(self, name, brithday,infos={}):
        self.name = name
        self.brithday = brithday
        self.infos=infos
        
    def __getattr__(self, item):
        return self.infos[item]  #通过这种逻辑实现对dict类型的直接访问

In [36]:
usera = Users(name = "a", brithday="nba", infos={"company":"huawei"})
usera.company

'huawei'

In [37]:
class Users:
    def __init__(self, name, brithday,infos={}):
        self.name = name
        self.brithday = brithday
        self.infos=infos
        
    def __getattr__(self, item):
        return self.infos[item]
    
    def __getattribute__(self, item):  # 访问任意属性前都需要过一遍这个函数，编程者可以通过此把握程序访问的一切入口
        return "asdasd"

## 8.3 属性描述符与属性查找过程

In [38]:
import numbers
class IntField:
    # 数据描述符
    def __get__(self, instance, owner):
        return self.value
        
    def __set__(self, instance, value):
        if not isinstance(value, numbers.Integral):
            raise ValueError("int value is needed")
        #......其他参数检查
        self.value = value
        
    def __delete__(self, instance):
        pass
    
class User:
    age = IntField()   # 以这种方式声明的属性，设置值时，调用IntField中 __set__ ，取值时调用IntField中__get__

In [41]:
usera = User()
usera.age = 30
print(usera.age)

30


In [42]:
# 查找过程不看了，烦

## 8.4 new和init区别

In [43]:
class User:
    def __new__(cls, *args, **kwargs):   #py2.4之后才有的，允许在创建实例前加逻辑
        # 参数传进来的 是类，而非实例
        # 第一个参数*arg接受用传值的方法传进的参数
        # 第二个参数**kwargs接受用  参数名=参数值  方法传进的参数
        return super().__new__(cls)  # 如果__new__方法不返回一个类，那么将不会调用init方法
    
    def __init__(self):  #__init__实在调用__new__，创建好对象后才调用的，用于给实例初始化的方法
        pass

## 8.5 自定义元类

In [46]:
def create_class(name):  # 此函数可以通过字符串控制动态创建类
    if name =="user":
        class User:
            def __str__(self):
                return "user"
        return User
    elif name =="company":
        class Company:
            def __str__(self):
                return "company"
        return Company


In [48]:
MyClass = create_class("user")
my_obj = MyClass()
print(my_obj)

user


In [49]:
# type可以用于获取对象类型，也可以用于动态创建类

def create_class(name):  # 此函数可以通过字符串控制动态创建类
    if name =="user":
        return type("User",(),{"name":"user"})  #第一个参数为类名（标识符），第二个参数此类的基类，第三个参数为初始化属性设置
    elif name =="company":
        return type("User",(),{})

    

In [51]:
MyClass = create_class("user")
my_obj = MyClass()
print(my_obj.name)

user


In [64]:
def say(self):
    print(self.name)

__main__.User

In [66]:
User = type("User",(),{"name":"user","say":say})   # 通过这种方式为动态类添加方法
my_obj = User()
my_obj.say()

user


In [67]:
class BaseClass:
    def anwser(self):
        return "i am a god"

User =  type("User",(BaseClass,),{"name":"user","say":say}) # 通过这种方式继承


In [68]:
my_obj = User()
print(my_obj.anwser())

i am a god


In [69]:
# 什么是元类？
# 元类是创建类的类，type就是一个元类
class MateClass(type):   #自定义元类需要继承type
    pass    

class User(metaclass=MateClass):  # 传递metaclass参数表示使用自定义元类创建该类
    pass

## 8.6 元类实现orm（略）